In [37]:
import os
import csv

In [38]:
def addrFile(tweeter,suffix):
    path=addrRoot+str(tweeter)+'/'
    if os.path.exists(path) is False:
         os.makedirs(path)
    address=path+tweeter+suffix
    return address  

In [39]:
#文件保存地址，改为你存放csv文件的完整地址
addrRoot='./data/'
#是否调试
isDebug=False

In [40]:
tweeter=input('data_name：')#武汉肺炎-搜狐新闻


In [41]:
#开始
startCollect=True


batch=0
dataDict={}
Title=[]
titleEixst=True
address= addrRoot + tweeter + '.csv'
print(address)

if os.path.exists(address) is True:
    fp = open(address,'r',newline='',encoding='utf-16')      
    reader=csv.reader(fp)
    for line in reader:
        if titleEixst is True:
            #将标题制成各个
            if startCollect is True:
                for item in line:
                    #print(item[-1])
                    if isDebug is True:
                        print(item)

                        
                    dataDict[item]=[]
                startCollect=False
            Title=line    
            if isDebug is True:
                print(line)
            titleEixst=False
        else:
            for no in range(len(line)):               
                if(Title[no].find('count')>=0):
                    if line[no].find('0万') > 0:
                        dataDict.get(Title[no]).append(infinity)
                    else:
                        if line[no] != '':
                            dataDict.get(Title[no]).append(eval(line[no]))
                    
                    #dataDict.get(Title[no]).append((line[no]))
                    
                else:
                    if line[no] != '':
                        dataDict.get(Title[no]).append((line[no]))
                if isDebug is True:
                    print(Title[no])
                    print(line[no])               
            


fp.close()

#print(dataDict)

./data/cs.csv


In [42]:
print(Title)

['转发created_at', '转发text', '转发reposts_count', '转发comments_count', '转发attitudes_count', '转发id', '转发screen_name', '转发description', '转发gender', '转发followers_count', '转发follow_count', '转发source_name']


### 读入数据

In [43]:
originData={}
retweetData={}

labels=[0 for i in range(4)]
cnLabels={}

labels[1]='reposts_count'
labels[2]='comments_count'
labels[3]='attitudes_count'
labels[0]='follow_count'

cnLabels[labels[1]]='转发量'
cnLabels[labels[2]]='评论量'
cnLabels[labels[3]]='点赞量'
cnLabels[labels[0]]='粉丝量'


for i in range(4):
    #print(choice+label)
    try:
        #originData[cnLabels[i]]=np.asarray(dataDict.get('原文'+labels[i])[::-1])
        retweetData[labels[i]]=np.asarray(dataDict.get('转发'+labels[i])[::-1])
    except:
        pass
    try:
        originData[labels[i]]=np.asarray(dataDict.get('原文'+labels[i])[::-1])
        #retweetData[cnLabels[i]]=np.asarray(dataDict.get('转发'+labels[i])[::-1])
    except:
        pass



In [44]:
import re
#工具类
class Tool:  
    repostEN=re.compile('//<a.*?>@(.*?)</a>:')#英文字符冒号
    repostCN=re.compile('//<a.*?>@(.*?)</a>：')#中文字符冒号
    @classmethod
    def findSource(cls,x):
        sourceName=''
        xEN=xCN=''
        
        xEN = re.findall(cls.repostEN,x)
        xCN = re.findall(cls.repostCN,x)
        
        
        #如果其中一者存在，另一者不存在，即返回该者
        if(len(xCN)==0 and len(xEN)>0):
            sourceName=xEN[0]
            #print(xEN[0])
        elif(len(xEN)==0 and len(xCN)>0):
            sourceName=xCN[0]
            #print(xCN[0]) 
        #若二者都存在，则返回第一位置字符串较小的
        elif(len(xEN)>0 and len(xCN)>0):
            sourceName=xCN[0]  if(len(xEN[0])>len(xCN[0])) else xEN[0]
                
        return sourceName

In [45]:
class Categories:
    
    def __init__(self):
        self.compose={}
    def add(self,name):
        self.compose[name]={}
        category=self.compose[name]
        category['value']=0 #记录被转次数
        category['target']={}
        category['source']={}
    def nameExist(self,name):
        if self.compose.get(name) is None:   
            return False
        else:
            return True
        
    def addTarget(self,sourceName,targetName):
        if self.nameExist(sourceName) is False:
            self.add(sourceName)
        if self.nameExist(targetName) is False:
            self.add(targetName)
        
        ##防止循环调用        
        if sourceName == targetName:
            #print(sourceName)
            return 
        if self.compose[targetName]['source'].get(sourceName) is not None:
            #print(sourceName)
            self.compose[targetName]['source'].pop(sourceName)
            
            
        if self.compose[targetName]['source'].get(sourceName) is None:
            self.compose[targetName]['source'][sourceName]=1
        else:
            self.compose[targetName]['source'][sourceName]+=1    
            
        
        if self.compose[sourceName]['target'].get(sourceName) is None:
            self.compose[sourceName]['target'][targetName]=1
        else:
            self.compose[sourceName]['target'][targetName]+=1
        self.compose[sourceName]['value']+=1    
            
    def countAll(self,name):        
        targets=self.compose[name]['target']
        if targets == {}:
            self.compose[name]['value']=0
        else:
            for targetName in targets:
                if self.compose[targetName]['target']=={}:
                    self.compose[targetName]['value']=0
                    #self.compose[name]['value']+=1
                else:
                    self.countAll(targetName)
                    self.compose[name]['value']+=self.compose[targetName]['value']
     #数据缺失补充 假定为转发原博主    
    def fillSource(self,tweeter):
        for item in list(self.compose.keys()):
            source=self.compose[item].get('source')
            if  (len(source))!=1 and item !=tweeter:
                self.addTarget(tweeter,item)
            #print(self.compose[item])
                    
            
        

In [47]:

import sys

 

sys.setrecursionlimit(9000000) #这里设置大一些


choice='转发'#'原文'#
categories=Categories()
txt=''



for name,text, sourceName in  zip(dataDict[choice+'screen_name'],dataDict[choice+'text'], dataDict[choice + 'source_name']):
    
    if categories.nameExist(name) is False:
        categories.add(name)
            
            
    if sourceName is not '':    
        categories.addTarget(sourceName,name)
        
    else:
        categories.addTarget(tweeter,name)

#categories.countAll(tweeter)#zwysgs')#处理所有，统计所有转发来源量        
categories.fillSource(tweeter)         
            
         

<>:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\hbh\AppData\Local\Temp\ipykernel_8212\299470273.py:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sourceName is not '':


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "e:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hbh\AppData\Local\Temp\ipykernel_8212\299470273.py", line 28, in <cell line: 28>
    categories.fillSource(tweeter)
  File "C:\Users\hbh\AppData\Local\Temp\ipykernel_8212\3885468635.py", line 58, in fillSource
    for item in self.compose:
RuntimeError: dictionary changed size during iteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "e:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "e:\Anaconda\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "e:\Anaconda\lib\site-packages\IPython\core\ultratb.py", line 1012, in structured_traceback
   

In [55]:
nodes=[]
links=[]
category=[]



for i in  categories.compose:
    value=categories.compose[i]['value']
    try:
        source=list(categories.compose[i]['source'])[0]
    except:
        source=tweeter
        #print(i)
    node={  "name":i,
            "symbolSize": 5,
            "draggable": "False",
            "value": value,
            "category": source
         }
            
           
    if value > 0:
        if i==tweeter:
            node["category"]=i
        #change node
        if source !=tweeter:
            #print(i)
            node["category"]=i
        symbolSize=value//10 
        if symbolSize>5:
            node['symbolSize']=symbolSize
        node['label']={
                "normal": {
                    "show": "True"
                }
            }
    
        
        
        #add link
        targets=categories.compose[i]['target']
        if targets != {}:
            for target in targets:
                links.append({'source':i,'target':target})
        
        
        #归入category
        category.append({'name':i})
    nodes.append(node)
        
        
    #try:
        #categories.compose[i]['target'][i] 
        
        #print(i)
        #print( categories.compose[i])
    #except:
        #pass
#print(nodes)
#print(links)
#print(category)

content='wuhan'
mid='4444444444444'
tweeter=tweeter

In [56]:
jsonData=[nodes,links,category,content,mid,tweeter]

In [57]:
import json

testFile=addrFile(tweeter,'.json')
with open(testFile,'w',encoding='utf-8') as file_obj:
    json.dump(jsonData,file_obj)
file_obj.close()

In [58]:
from pyecharts import options as opts
from pyecharts.charts import Geo, Page, Bar ,Graph
from pyecharts.faker import Collector, Faker
from pyecharts.globals import CurrentConfig,NotebookType#ChartType, SymbolType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

def graph_weibo() -> Graph:
    with open(testFile, "r", encoding="utf-8") as f:
        j = json.load(f)
        nodes, links, categories, cont, mid, userl = j
    c = (
        Graph()
        .add(
            "肺炎",
            nodes,
            links,
            categories,
            repulsion=50,
             #layout="none",
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
            label_opts=opts.LabelOpts(is_show=False),#True),#
        )
        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False),#True),#
            title_opts=opts.TitleOpts(title=tweeter+"微博转发关系图"),
        )
    )
    print(userl)
    return c



In [59]:
c = graph_weibo()

cs


In [60]:
c.load_javascript()

In [61]:
c.render_notebook()

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Geo, Page, Bar ,Graph
from pyecharts.faker import Collector, Faker
from pyecharts.globals import CurrentConfig,NotebookType#ChartType, SymbolType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

C = Collector()

@C.funcs
def graph_weibo() -> Graph:
    with open(testFile, "r", encoding="utf-8") as f:
        j = json.load(f)
        nodes, links, categories, cont, mid, userl = j
    c = (
        Graph()
        .add(
            "肺炎",
            nodes,
            links,
            categories,
            repulsion=50,
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
            label_opts=opts.LabelOpts(is_show=False),#True),#
        )
        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False),#True),#
            title_opts=opts.TitleOpts(title=tweeter+"BiliBili转发关系图"),
        )
    )
    print(userl)
    return c

Page().add(*[fn() for fn, _ in C.charts]).render()